In [1]:
# !jupyter nbextension list

In [2]:
# !jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !jupyter nbextension enable --py gmaps 
# %matplotlib inline 
# !jupyter nbextension install --py --symlink --user gmaps
# !jupyter nbextension enable --py --user gmaps

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
cities_df = pd.read_csv("openweathermaps.csv")
cities_df

,cloudiness,country,date,humidity,max_temp,lat,lng,wind_speed,city_name
0,100,GB,1631720677,86,57.11,60.3964,-1.3530,10.67,Brae
1,75,US,1631720414,75,75.52,22.0752,-159.3190,16.11,Kapaa
2,0,MZ,1631720679,61,78.31,-19.6094,34.7431,8.57,Dondo
3,20,AR,1631720620,39,44.65,-51.6226,-69.2181,43.73,Río Gallegos
4,87,US,1631720682,82,81.41,41.8007,-73.1212,1.01,Torrington
...,...,...,...,...,...,...,...,...,...
542,100,MX,1631721493,65,85.33,20.8667,-90.4000,7.87,Celestún
543,100,PY,1631721495,78,64.85,-23.3428,-57.0597,6.33,Horqueta
544,77,RU,1631721497,90,43.86,65.5333,72.5167,7.96,Nadym
545,75,ES,1631721498,73,76.75,28.6585,-17.9182,6.91,Los Llanos de Aridane


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(g_key)

In [48]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=3)
locations = cities_df[["lat", "lng"]]
humidity = cities_df["humidity"]
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
heatmap.max_intensity = 2
heatmap.point_radius = 3
heatmap.gradient = ['white', 'gray']
fig.add_layer(heatmap)
# plt.savefig("heat_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
nd_cities_df = cities_df.loc[(cities_df['wind_speed'] <= 10) | (cities_df['cloudiness'] ==0) | (cities_df['max_temp'] >=70) | (cities_df['max_temp'] <=80)].dropna()
nd_cities_df

,cloudiness,country,date,humidity,max_temp,lat,lng,wind_speed,city_name
0,100,GB,1631720677,86,57.11,60.3964,-1.3530,10.67,Brae
1,75,US,1631720414,75,75.52,22.0752,-159.3190,16.11,Kapaa
2,0,MZ,1631720679,61,78.31,-19.6094,34.7431,8.57,Dondo
3,20,AR,1631720620,39,44.65,-51.6226,-69.2181,43.73,Río Gallegos
4,87,US,1631720682,82,81.41,41.8007,-73.1212,1.01,Torrington
...,...,...,...,...,...,...,...,...,...
542,100,MX,1631721493,65,85.33,20.8667,-90.4000,7.87,Celestún
543,100,PY,1631721495,78,64.85,-23.3428,-57.0597,6.33,Horqueta
544,77,RU,1631721497,90,43.86,65.5333,72.5167,7.96,Nadym
545,75,ES,1631721498,73,76.75,28.6585,-17.9182,6.91,Los Llanos de Aridane


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = nd_cities_df.loc[:,['lat', 'lng', 'city_name', 'country']]
hotel_df['hotel name'] = " "
hotel_df

,lat,lng,city_name,country,hotel name
0,60.3964,-1.3530,Brae,GB,
1,22.0752,-159.3190,Kapaa,US,
2,-19.6094,34.7431,Dondo,MZ,
3,-51.6226,-69.2181,Río Gallegos,AR,
4,41.8007,-73.1212,Torrington,US,
...,...,...,...,...,...
542,20.8667,-90.4000,Celestún,MX,
543,-23.3428,-57.0597,Horqueta,PY,
544,65.5333,72.5167,Nadym,RU,
545,28.6585,-17.9182,Los Llanos de Aridane,ES,


In [38]:
params = {
    "types": "hotel",
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key
}

for index, row in hotels_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    city_name = row["city_name"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()

    try:
        hotel_df.loc[index, "hotel name"] = hotel_name["results"][0]["name"]
    except:
        print ("missing field")


missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missing field
missin

In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>name</dt><dd>{hotel name}</dd>
<dt>city</dt><dd>{city_name}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [47]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display Map
# plt.savefig("hotel_map.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>